# Initialisation

In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

In [2]:
import sys
import os

py_file_location = '../'
home_directory = '../'

sys.path.append(os.path.abspath(py_file_location))
from model.model_class.environment import *

from model.model_class import GNN

SEED = 42

# Data Preparation

In [3]:
gnn_train_data = pd.read_parquet('../data/curated/ML_data/gnn_train_data.parquet')
gnn_val_data = pd.read_parquet('../data/curated/ML_data/gnn_val_data.parquet')
gnn_test_data = pd.read_parquet('../data/curated/ML_data/gnn_test_data.parquet')

SA2_gnn_data = pd.read_parquet('../data/curated/ML_data/SA2_gnn_data.parquet')
SA2_gnn_data['Station Name'] = SA2_gnn_data['Station Name'].apply(lambda x: '(SA2)'+x)
station_inference_gnn_data = pd.read_parquet('../data/curated/ML_data/station_inference_gnn_data.parquet')

station_inference_gnn_data = station_inference_gnn_data.rename({'Station_Name': 'Station Name'}, axis=1)
inference_data = pd.concat([SA2_gnn_data, station_inference_gnn_data], axis=0)
inference_data.drop(columns=['Station_Na'], axis=1, inplace=True)
inference_data = inference_data.rename({'Station Name': 'Station_Name'}, axis=1)

In [4]:
# open npy
station_weights_matrix = np.load('../data/curated/ML_data/station_weights_matrix.npy')
SA2_weights_matrix = np.load('../data/curated/ML_data/station_weights_withSA2_matrix.npy')

with open('../data/curated/ML_features/station_weights_withSA2.json', 'r') as f:
    station_weights_withSA2 = json.load(f)

with open('../data/curated/ML_features/station_weights.json', 'r') as f:
    station_weights = json.load(f)

In [5]:
# edit SA2_weights_matrix matrix: columns after 223 masked out to 0
SA2_weights_matrix[:, 223:] = 0

In [6]:
geospatial_features = ['log_Total_Demand']
non_geospatial_features = ['Weekday', 'mean_rainfall_value', 'has_school',
       'has_sport_facility', 'has_shopping_centre', 'has_hospital',
       'total_population', ' med_rent_weekly_c2021',
       ' med_mortg_rep_mon_c2021', ' med_person_inc_we_c2021',
       ' med_famly_inc_we_c2021']
label_columns = ['log_Total_Demand']

In [7]:
def DataFactory(raw_dataset, geospatial_features, non_geospatial_features, label_columns, stations_index, inference = False):

    """ Data Factory of GNN """
    
    geospatial_x_batches = []
    non_geospatial_x_batches = []
    y_batches = []
    masks = []

    if inference:
        groupby_column = 'Weekday'
    else:
        groupby_column = 'Business_Date'

    for day, daily_df in tqdm(raw_dataset.groupby(groupby_column)):

        geospatial_x = np.zeros([len(stations_index), len(geospatial_features)])
        y = np.zeros([len(stations_index), len(label_columns)])
        mask = np.zeros([len(stations_index), 1])
        non_geospatial_x = np.zeros([len(stations_index), len(non_geospatial_features)])

        daily_df.set_index('Station_Name', inplace=True)

        for station in daily_df.index:

            geospatial_x[stations_index[station]] = daily_df.loc[station][geospatial_features] # todo inference. 
            if not inference:
                y[stations_index[station]] = daily_df.loc[station][label_columns]
            mask[stations_index[station]] = 1
            non_geospatial_x[stations_index[station]] = daily_df.loc[station][non_geospatial_features]
                
        geospatial_x = np.nan_to_num(geospatial_x)
        geospatial_x_batches.append(geospatial_x)
        y_batches.append(y)
        masks.append(mask.flatten())

        non_geospatial_x_batches.append(non_geospatial_x)

        
    return geospatial_x_batches, non_geospatial_x_batches, y_batches, masks

In [7]:
train_geospatial_X_batches, train_non_geospatial_X_batches, train_y_batches, train_masks = DataFactory(gnn_train_data, geospatial_features, non_geospatial_features, label_columns, station_weights)
val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks = DataFactory(gnn_val_data, geospatial_features, non_geospatial_features, label_columns, station_weights)
test_geospatial_X_batches, test_non_geospatial_X_batches, test_y_batches, test_masks = DataFactory(gnn_test_data, geospatial_features, non_geospatial_features, label_columns, station_weights)

  0%|          | 0/382 [00:00<?, ?it/s]

100%|██████████| 82/82 [00:14<00:00,  5.59it/s]


In [8]:
all_inference_geospatial_X_batches, all_inference_non_geospatial_X_batches, all_inference_y_batches, all_inference_masks = DataFactory(inference_data, geospatial_features, non_geospatial_features, label_columns, station_weights_withSA2, inference = True)

100%|██████████| 2/2 [00:00<00:00,  2.94it/s]


# Training

In [9]:
class GNN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_heads = 8
    dropout = 0.1
    n_gnn_layers = 1
    activation = nn.ReLU()
    res_learning = False
    bottleneck = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    epochs = 32
    lr = 1e-3
    patience = 5
    loss = nn.MSELoss()
    validation_loss = nn.MSELoss()
    alpha = 0.1
    scheduler = True
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    spatial_input_dim = 1
    nonspatial_input_dim = 11
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory
    name = f'AGNN'

model1 = GNN(GNN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model1.fit(train_geospatial_X_batches, train_non_geospatial_X_batches, train_y_batches, train_masks, val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix)
print('\n\n')

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model1.load()
model1.eval(val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix, best_epoch, evaluation_mode = True)
model1.eval(test_geospatial_X_batches, test_non_geospatial_X_batches, test_y_batches, test_masks, station_weights_matrix, best_epoch, evaluation_mode = True)

  0%|          | 0/382 [00:00<?, ?it/s]

100%|██████████| 382/382 [00:16<00:00, 22.51it/s]


 Epoch 1 Train | Loss:  0.0931 | R2:  0.8988| MSE:  0.0928 | RMSE:  0.3047 | MAE:  0.1941 


100%|██████████| 82/82 [00:00<00:00, 83.27it/s]
/Users/tg.chenny/Desktop/1. University/2. Masters/9. Geospatial Data Analysis/A4/GEOM90006-A4_TrainDemandInGreaterMelbourne__Groupwork__Py/model/model_class/__template__.py:193: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  val_y_tensor = torch.FloatTensor(val_y).to(self.device)


Epoch 1 Val | Loss:  0.0182 | R2:  0.9818| MSE:  0.0182 | RMSE:  0.1348 | MAE:  0.1072 


...

Epoch 11 Val | Loss:  0.0044 | R2:  0.9956| MSE:  0.0044 | RMSE:  0.0660 | MAE:  0.0501 





100%|██████████| 82/82 [00:00<00:00, 115.10it/s]


Epoch 8 Val | Loss:  0.0021 | R2:  0.9979| MSE:  0.0021 | RMSE:  0.0459 | MAE:  0.0332 


100%|██████████| 82/82 [00:00<00:00, 125.49it/s]

Epoch 8 Val | Loss:  0.0022 | R2:  0.9975| MSE:  0.0022 | RMSE:  0.0470 | MAE:  0.0343 


In [10]:
class GNN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_heads = 8
    dropout = 0.1
    n_gnn_layers = 2
    activation = nn.ReLU()
    res_learning = False
    bottleneck = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    epochs = 32
    lr = 1e-3
    patience = 5
    loss = nn.MSELoss()
    validation_loss = nn.MSELoss()
    alpha = 0.1
    scheduler = True
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    spatial_input_dim = 1
    nonspatial_input_dim = 11
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory
    name = f'AGNN_2layer'

model2 = GNN(GNN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model2.fit(train_geospatial_X_batches, train_non_geospatial_X_batches, train_y_batches, train_masks, val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix)
print('\n\n')

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model2.load()
model2.eval(val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix, best_epoch, evaluation_mode = True)
model2.eval(test_geospatial_X_batches, test_non_geospatial_X_batches, test_y_batches, test_masks, station_weights_matrix, best_epoch, evaluation_mode = True)

  0%|          | 0/382 [00:00<?, ?it/s]

100%|██████████| 382/382 [00:23<00:00, 16.44it/s]


 Epoch 1 Train | Loss:  0.1195 | R2:  0.8701| MSE:  0.1191 | RMSE:  0.3452 | MAE:  0.2217 


100%|██████████| 82/82 [00:01<00:00, 63.24it/s]


Epoch 1 Val | Loss:  0.0248 | R2:  0.9751| MSE:  0.0248 | RMSE:  0.1576 | MAE:  0.1200 


...

Epoch 14 Val | Loss:  0.0057 | R2:  0.9943| MSE:  0.0057 | RMSE:  0.0753 | MAE:  0.0541 





100%|██████████| 82/82 [00:01<00:00, 53.20it/s]


Epoch 13 Val | Loss:  0.0044 | R2:  0.9956| MSE:  0.0044 | RMSE:  0.0663 | MAE:  0.0538 


100%|██████████| 82/82 [00:01<00:00, 56.99it/s]


Epoch 13 Val | Loss:  0.0043 | R2:  0.9952| MSE:  0.0043 | RMSE:  0.0659 | MAE:  0.0533 


# Inference

In [171]:
os.makedirs('../output', exist_ok=True)

# read in y scale
with open('../data/curated/ML_data/y_scaler_gnn.pickle', 'rb') as f:
    y_scaler_gnn = pickle.load(f)

In [281]:
all_predictions = model2.predict(all_inference_geospatial_X_batches, all_inference_non_geospatial_X_batches, all_inference_masks, SA2_weights_matrix)
all_predictions = np.array(all_predictions).flatten()

all_predictions_df = pd.DataFrame({'Station Name': list(station_weights_withSA2.keys()) * 2,
              'Predicted_Log_Total_Demand': all_predictions,
              'Weekday': [0 for _ in range(len(station_weights_withSA2))] + [1 for _ in range(len(station_weights_withSA2))]})
all_predictions_df['Unscaled_Predicted_Log_Total_Demand'] = y_scaler_gnn.inverse_transform(all_predictions_df['Predicted_Log_Total_Demand'].values.reshape(-1, 1))
all_predictions_df['Unscaled_Predicted_Total_Demand'] = np.exp(all_predictions_df['Unscaled_Predicted_Log_Total_Demand'])
all_predictions_df.to_csv('../output/agnn2_predictions.csv', index=False)

all_predictions_df

100%|██████████| 2/2 [00:00<00:00, 29.04it/s]


,Station Name,Predicted_Log_Total_Demand,Weekday,Unscaled_Predicted_Log_Total_Demand,Unscaled_Predicted_Total_Demand
0,Aircraft,2.885838,0,12.258663,210799.561393
1,Alamein,2.880375,0,12.248887,208748.728791
2,Albion,2.902173,0,12.287896,217052.903606
3,Alphington,2.916082,0,12.312788,222523.576722
4,Altona,2.900282,0,12.284512,216319.642428
...,...,...,...,...,...
879,(SA2)Whittlesea,0.019191,1,7.128467,1246.964479
880,(SA2)Wollert,-0.413111,1,6.354812,575.254399
881,(SA2)Wyndham Vale - North,0.147060,1,7.357303,1567.602840
882,(SA2)Wyndham Vale - South,0.052260,1,7.187648,1322.987988


In [286]:
# want 5/7 * weekday = 1 + 2/7 * weekend = 0
weekday_weekend_scaled_predictions = all_predictions_df[['Station Name', 'Weekday', 'Unscaled_Predicted_Total_Demand']].groupby('Station Name').apply(lambda x: 5/7 * x[x['Weekday'] == 1]['Unscaled_Predicted_Total_Demand'].values[0] + 2/7 * x[x['Weekday'] == 0]['Unscaled_Predicted_Total_Demand'].values[0]).reset_index().rename({0: 'Predicted_Total_Demand'}, axis=1).loc[:218]
weekday_weekend_scaled_predictions.to_csv('../output/agnn2_predictions_weekday_weekend_scaled.csv', index=False)

/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_25905/2378514990.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weekday_weekend_scaled_predictions = all_predictions_df[['Station Name', 'Weekday', 'Unscaled_Predicted_Total_Demand']].groupby('Station Name').apply(lambda x: 5/7 * x[x['Weekday'] == 1]['Unscaled_Predicted_Total_Demand'].values[0] + 2/7 * x[x['Weekday'] == 0]['Unscaled_Predicted_Total_Demand'].values[0]).reset_index().rename({0: 'Predicted_Total_Demand'}, axis=1).loc[:218]


# Feature Analysis

Last 2 tuples correspond to weights for: 'Nearby Train Demand Aggregate', 'Weekday', 'PublicHoliday', 'mean_rainfall_value', 'has_school',
       'has_sport_facility', 'has_shopping_centre', 'has_hospital',
       'total_population', ' med_rent_weekly_c2021',
       ' med_mortg_rep_mon_c2021', ' med_person_inc_we_c2021',
       ' med_famly_inc_we_c2021'

In [232]:
# A_GNN features
class GNN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_heads = 8
    dropout = 0.1
    n_gnn_layers = 1
    activation = nn.ReLU()
    res_learning = False
    bottleneck = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    epochs = 32
    lr = 1e-3
    patience = 5
    loss = nn.MSELoss()
    validation_loss = nn.MSELoss()
    alpha = 0.1
    scheduler = True
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    spatial_input_dim = 1
    nonspatial_input_dim = 11
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory
    name = f'AGNN'

model1 = GNN(GNN_config) # initialise the model

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model1.load()

for parameters in model1.model.parameters():
    print(parameters)

Parameter containing:
tensor([[ 0.7480],
        [ 0.8614],
        [-0.2957],
        [ 0.9152],
        [-0.2137],
        [ 0.1957],
        [-0.5804],
        [ 0.6436],
        [ 0.8857],
        [-0.7353],
        [ 0.9511],
        [ 0.1574],
        [ 0.7965],
        [ 0.1313],
        [ 0.4371],
        [-0.1369],
        [ 0.7657],
        [ 0.1818],
        [-0.4912],
        [ 0.2355],
        [-0.4309],
        [-0.0742],
        [-0.4500],
        [ 0.7314],
        [-0.7856],
        [-0.5282],
        [-0.1489],
        [-0.6174],
        [ 0.0915],
        [-0.9699],
        [ 0.9564],
        [-0.8120],
        [ 0.7544],
        [ 0.0969],
        [-0.3586],
        [ 0.6360],
        [ 0.1512],
        [ 0.8130],
        [ 0.0952],
        [-0.2518],
        [ 0.3636],
        [-0.1584],
        [ 0.4639],
        [ 0.9584],
        [ 0.6344],
        [-0.4793],
        [ 0.6207],
        [ 0.1576],
        [ 0.5306],
        [-0.6351],
        [-0.9789],
        [

In [198]:
# A_GNN 2 layers
class GNN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_heads = 8
    dropout = 0.1
    n_gnn_layers = 2
    activation = nn.ReLU()
    res_learning = False
    bottleneck = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    epochs = 32
    lr = 1e-3
    patience = 5
    loss = nn.MSELoss()
    validation_loss = nn.MSELoss()
    alpha = 0.1
    scheduler = True
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    spatial_input_dim = 1
    nonspatial_input_dim = 11
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory
    name = f'AGNN_2layer'

model2 = GNN(GNN_config) # initialise the model

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model2.load()

for parameters in model2.model.parameters():
    print(parameters)

Parameter containing:
tensor([[ 0.7566],
        [ 0.8149],
        [-0.2277],
        [ 0.9544],
        [-0.1765],
        [ 0.1935],
        [-0.5477],
        [ 0.6236],
        [ 0.8285],
        [-0.7118],
        [ 0.8722],
        [ 0.0569],
        [ 0.7579],
        [ 0.1298],
        [ 0.4313],
        [-0.1354],
        [ 0.7726],
        [ 0.1683],
        [-0.2830],
        [ 0.2171],
        [-0.3417],
        [-0.0364],
        [-0.4924],
        [ 0.6870],
        [-0.8058],
        [-0.3944],
        [-0.1358],
        [-0.6120],
        [ 0.0905],
        [-1.0056],
        [ 0.9355],
        [-0.8550],
        [ 0.7969],
        [ 0.0743],
        [-0.2128],
        [ 0.6609],
        [ 0.1494],
        [ 0.7959],
        [ 0.0850],
        [-0.1499],
        [ 0.3227],
        [-0.0836],
        [ 0.4428],
        [ 0.9318],
        [ 0.6178],
        [-0.4134],
        [ 0.6152],
        [ 0.1558],
        [ 0.5816],
        [-0.5914],
        [-0.9899],
        [